In [9]:
import numpy as np
import os
from mxnet import nd, gpu, gluon, autograd
import mxnet as mx
import pandas as pd
from mxnet import gluon
import gc 

from gluonts.env import env

# env._push(use_tqdm=False)

import json
from gluonts.dataset.common import load_datasets
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.dataset.rolling_dataset import NumSplitsStrategy, StepStrategy, generate_rolling_dataset, truncate_features
from gluonts.model.seq2seq import MQRNNEstimator
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.distribution import PiecewiseLinearOutput
from gluonts.mx.trainer import Trainer
from gluonts.mx.trainer.model_averaging import SelectNBestMean
from gluonts.mx.trainer.model_iteration_averaging import Alpha_Suffix
from gluonts.evaluation import Evaluator
from pathlib import Path
from gluonts.evaluation.backtest import make_evaluation_predictions
from tqdm.notebook import tqdm


from gluonts.env import env

/home/pedro/miniconda/envs/mxnet_clean/lib/python3.7/site-packages/gluonts/json.py:46: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  "Using `json`-module for json-handling. "


In [10]:
# activate whatever environment. 
# cd into the gluonts-local directory
# pip install . 

In [11]:
from quantile_forecast import *

In [5]:
def make_trainer(config):
    trainer = Trainer(epochs=config["epochs"],
                      learning_rate=config["learning_rate"],
                      learning_rate_decay_factor=config["learning_rate_decay_factor"], 
                      weight_decay=config["weight_decay"],
                      ctx=mx.cpu())
    return trainer

In [7]:
dataset_path = "/home/pedro/pedro_code/deep-traffic/fresh/data/data_to_tune/chic/chic.gluonts"
model_path = "./models/chic_mqrnn"
config_path = "./configs/test/config_chic_5_nonmonot.json"
quantile_boundary = 1e-3
# num_pieces = 30
monotonic = False
prediction_length = 1

with open(config_path) as json_file:
        config = json.load(json_file)
        print(config)
config['epochs'] = 5
num_quantiles = config["num_quantiles"]
dataset = load_dataset(dataset_path)
quantiles = make_quantiles(dataset, num_quantiles, quantile_boundary)

trainer = make_trainer(config)
estimator = make_mqrnn_estimator(trainer, quantiles, monotonic, dataset.metadata.freq, prediction_length)
predictor = make_predictor(estimator, dataset)

{'epochs': 474, 'learning_rate': 0.00023946721735196308, 'learning_rate_decay_factor': 0.6898245880780415, 'weight_decay': 5.036032509222127e-09, 'num_quantiles': 23}
learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


/home/pedro/miniconda/envs/mxnet_clean/lib/python3.7/site-packages/gluonts/transform/split.py:36: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  return _shift_timestamp_helper(ts, ts.freq, offset)
/home/pedro/miniconda/envs/mxnet_clean/lib/python3.7/site-packages/gluonts/transform/feature.py:352: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  self._min_time_point, self._max_time_point, freq=start.freq


In [8]:
print("here")
metrics, forecast, tsit = evaluate(predictor, dataset, quantiles, prediction_length)
print(metrics)

here


/home/pedro/miniconda/envs/mxnet_clean/lib/python3.7/site-packages/gluonts/dataset/common.py:317: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  timestamp = pd.Timestamp(string, freq=freq)
/home/pedro/miniconda/envs/mxnet_clean/lib/python3.7/site-packages/gluonts/dataset/common.py:320: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  if isinstance(timestamp.freq, Tick):
/home/pedro/miniconda/envs/mxnet_clean/lib/python3.7/site-packages/gluonts/dataset/common.py:322: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  timestamp.floor(timestamp.freq), timestamp.freq
/home/pedro/miniconda/envs/mxnet_clean/lib/python3.7/site-packages/gluonts/dataset/common.py:322: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  timestamp.floor(timestamp.freq), timestamp.freq
/home/pedro/miniconda/envs/mxnet_clean/lib/python

  0%|          | 0/128 [00:00<?, ?it/s]

{'crps': 0.04201652236507181, '60cover': 0.5234375, '80cover': 0.8828125, '95cover': 0.921875}


Loading dataset vadim
vadimloaded


In [159]:
# SOURCE_PATH = "/mnt/spacious/shared/vadim_all_sensors.csv"
# def get_dataframe(source_path):
#     df = pd.read_csv(source_path)
#     df["ds"] = pd.to_datetime(df["ds"])
#     df["target"] = df.groupby("DeviceId").Occupancy.fillna(method="ffill", limit=5)
#     df["sensor"], _ = pd.factorize(df["DeviceId"])
#     return df
# test_breaks = pd.read_csv('/mnt/spacious/shared/vadim_outlier_test.csv', usecols= ['ds',"sensor"])


SOURCE_PATH =  '/mnt/spacious/shared/data_dir1_may2020.csv'

def get_dataframe(source_path):
    df = pd.read_csv(source_path)
    df["ds"] = None
    for sensor in df["sensor"].unique():
        df.loc[df["sensor"] == sensor, "ds"] = pd.date_range(start='1/1/2008', end='3/1/2009', freq="1min")[:-1]
    df["ds"] = pd.to_datetime(df["ds"])
    df["target"] = df.groupby("sensor").dens_input_nan.fillna(method="ffill", limit=5)
    return df

test_breaks = pd.read_csv('/mnt/spacious/shared/peter_outlier_test.csv', usecols= ['ds',"sensor"])
df = get_dataframe(SOURCE_PATH)

### Making Outliers Data Set

In [31]:
def make_segments_chicago(df, test_breaks, diffed=False, drift_thresh=360000000.0*4, min_target_length=288//5):
    segments = []
    item_id = 0
    for s in tqdm(df.sensor.unique()):
        target_sequence = (df.loc[df.sensor == s, "target"]).to_numpy()
        temp_df = df[df.sensor == s].reset_index()
        outlier_idx = temp_df[temp_df.ds.isin(test_breaks[test_breaks.sensor == s].ds)].index
        target_sequence -= np.min(target_sequence)
        target_sequence /= np.max(target_sequence)
        ds_sequence =  (df.loc[df.sensor == s, "ds"]).to_numpy()[1:]
        start_idx = 0
        end_idx = 0
        diff_seq = np.diff(ds_sequence).astype(int)
        m = np.median(diff_seq)
#         print(outlier_idx)
        
        for count, end_idx in enumerate(outlier_idx):
            start_idx = 1000
            while np.isnan(target_sequence[end_idx-start_idx:end_idx]).sum()>0:
                start_idx -= 1
                if start_idx == end_idx:
                    print("here")
                    break

            t_seq = target_sequence[end_idx-start_idx:end_idx].astype(np.float32)
            
            seg = dict(
                start=ds_sequence[end_idx-start_idx], 
                end=ds_sequence[end_idx],
                target= t_seq,
                feat_static_cat=np.array([s], dtype=int),
                item_id = item_id,
                real_time = ds_sequence[:end_idx],
            )
            
            if ((end_idx == outlier_idx).sum()) == 1:
                segments.append(seg)
                
        item_id += 1
    segments = [s for s in segments if len(s["target"]) > min_target_length]

    return segments, item_id


In [27]:
df[df.sensor==1].reset_index().index

RangeIndex(start=0, stop=393722, step=1)

In [97]:
def make_segments_milwaukee(df, test_breaks, diffed=False, drift_thresh=360000000.0*4, min_target_length=288//4):
    segments = []

    item_id = 0

    for s in tqdm(df.sensor.unique()):
        target_sequence = (df.loc[df.sensor == s, "target"]).to_numpy()[::1] 
        target_sequence -= np.nanmin(target_sequence)
        target_sequence /= np.nanmax(target_sequence)
        ds_sequence =  (df.loc[df.sensor == s, "ds"]).to_numpy()[::1]
        outlier_idx = df[(df.sensor == s) & (df.ds.isin(test_breaks[test_breaks.sensor == s].ds))].index-612000*(s-1)
    
        for count, end_idx in enumerate(outlier_idx):
            start_idx = 1000
            while np.isnan(target_sequence[end_idx-start_idx:end_idx]).sum()>0:
                start_idx -= 1
                if start_idx == end_idx:
                    print("here")
                    break

            t_seq = target_sequence[end_idx-start_idx:end_idx].astype(np.float32)
            
            seg = dict(
                start=ds_sequence[end_idx-start_idx], 
                end=ds_sequence[end_idx],
                target= t_seq,
                feat_static_cat=np.array([s], dtype=int),
                item_id = item_id,
                real_time = ds_sequence[:end_idx],
            )
            
            if ((end_idx == outlier_idx).sum()) == 1:
                segments.append(seg)
                
        item_id += 1   
    segments = [s for s in segments if len(s["target"]) > min_target_length]
    return segments, item_id

In [32]:
segments, item_id = make_segments_chicago(df, test_breaks)

  0%|          | 0/21 [00:00<?, ?it/s]

In [36]:
from preprocessing import *
save_gluon("/home/pedro/pedro_code/deep-traffic/fresh/data/chicago_outlier.gluonts", segments, segments, '5T')

### NAIVE


In [162]:

# df['occ'] = (df.Occupancy - df.Occupancy.min())/df.Occupancy.max()
df['occ'] = (df.target - df.target.min())/df.target.max()
df["WeekDay"] = df.ds.dt.weekday

time_list = df.ds.dt.hour.value_counts().index     
# sensors = range(21)
sensors = range(1,19)
quantiles = [0.001, 0.005, 0.025, 0.05, 0.10, 0.25, 0.33, 0.5, 0.66, 0.75, 0.90, 0.95, 0.975, 0.995, 0.999]
# df_train = df[df.ds.dt.year < 2012]
# df_test = df[df.ds.dt.year >= 2012]
df_train = df[df.ds.dt.year <= 2008]
df_test = df[df.ds.dt.year >= 2009]
forecasts = {}

for weekday in range(7):
    for time in range(24):
        input_to_forecast = (weekday, time)
        current_df = df_train[(df.WeekDay == weekday) & (df.ds.dt.hour == time)].occ
        forecast = current_df.quantile(quantiles)
        forecasts[input_to_forecast] = forecast
#         print(input_to_forecast)
        count+=1

/home/pedro/miniconda/envs/mxnet_clean/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [163]:
y_actual = df_test.occ.values
y_actual_outliers = df_test[df_test.ds.isin(test_breaks.ds)].occ.values

In [164]:
inputs_for_forecast = [(weekday, hour) for weekday, hour in zip(df_test.WeekDay.values, df_test.ds.dt.hour.values)]
y_pred = np.array([forecasts[input].values for input in inputs_for_forecast])

inputs_for_outliers = [(weekday, hour) for weekday, hour in zip(pd.to_datetime(test_breaks.ds).dt.weekday, pd.to_datetime(test_breaks.ds).dt.hour.values)]
y_pred_outliers = np.array([forecasts[input].values for input in inputs_for_outliers])

In [165]:
# from quantile_forecast import *

def crps(tau, F_inv, x, n_steps=10000, miny=-1, maxy=100):
    F = interpolate.interp1d(F_inv, tau, kind="linear", bounds_error=False, fill_value=(0, 1))
    y_below = np.linspace(miny, x, n_steps)
    y_above = np.linspace(x, maxy, n_steps)
    
    error_below = np.power(F(y_below), 2)*np.ptp(y_below)/y_below.shape[0]
    error_above = np.power(1 - F(y_above), 2)*np.ptp(y_above)/y_above.shape[0]
    return (np.sum(error_below)  + np.sum(error_above))

def average_crps(quantiles, y_pred, y_actual):
    total = 0
    count = 0
    nb_samples = y_pred.shape[0]
    for i in tqdm(range(nb_samples)):
        c = crps(quantiles, y_pred[i, :], y_actual[i])
        if not np.isnan(c):
            total += c
            count += 1
    return total/max(count, 1e-16)



In [166]:
average_crps(quantiles, y_pred[::110], y_actual[::110])

  0%|          | 0/13903 [00:00<?, ?it/s]

0.017645062286443684

In [167]:
average_crps(quantiles, y_pred_outliers[::110], y_actual_outliers[::110])

  0%|          | 0/39 [00:00<?, ?it/s]

0.045051142659577326

In [131]:
y_pred[::100].shape

(28902, 15)

In [168]:
def coverage(quantiles, pred, y_actual):
    print("target | actual")
    target = []
    actual = []
    for i in range(len(quantiles)//2):
        lower_pred = pred[:, i]
        upper_pred = pred[:, -(i + 1)]
        target.append(quantiles[-(i + 1)] - quantiles[i])
        actual.append(np.mean(np.logical_and(lower_pred <= y_actual, y_actual <= upper_pred)))
#         print(f"{target:5.4f} | {actual:5.4f} | {(target-actual)/target:5.4f}")

    return (target, actual)

In [169]:
coverage(quantiles, y_pred, y_actual)

target | actual


([0.998, 0.99, 0.95, 0.8999999999999999, 0.8, 0.5, 0.33],
 [0.945000261560996,
  0.9422832967147939,
  0.923825591127851,
  0.891197164678803,
  0.8307471489851433,
  0.6032780131826742,
  0.4790640039757271])